<a href="https://colab.research.google.com/github/Leotzu/basic_RL_Q-Learning/blob/master/q_learning_shortest_path.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np

In [52]:
# Train the agent in the environment
def train(alpha, gamma, iterations, location_to_state, state_to_location, actions, reward_env, check_point):

    # initialize Q table with all zeros
    q_table = np.zeros_like(reward_env)

    for i in range(1, iterations+1):
        # pick a random current state 
        current_state = np.random.randint(0, 9)
        playable_actions = []

        '''
        Iterate through the rewards matrix to get the states which
        are directly reachable from the randomly chosen current state.
        Then add those states to a list called playable_actions.
        '''
        for j in range(9):
            if reward_env[current_state, j] > 0:
                playable_actions.append(j)

        # if current location has no out-going edges, skip
        if len(playable_actions) == 0:
            continue

        # randomly choose the one of the reachable states as the next state
        next_state = np.random.choice(playable_actions)

        # Bellman's Equation (finding temporal difference)
        q_table[current_state, next_state] += alpha * (reward_env[current_state, next_state] \
                                           + gamma * q_table[next_state, np.argmax(q_table[next_state,])] \
                                           - q_table[current_state, next_state])
        
        # Print out the Q-table at every check_point
        if check_point > 0:
            if i % check_point == 0:
                print(f'Q-table at iteration {i}:\n {q_table}\n')
        
    return q_table

# function to print the shortest path
def get_optimal_route(start_location, end_location, q_table, location_to_state, state_to_location):
    # initialize the route and next_location
    route = [start_location]
    next_location = start_location
    current_location = start_location

    # append the next location to the route list then print that list
    while next_location != end_location:
        current_state = location_to_state[current_location]

        # to avoid infinite loops, remove reflexive reward at current_state
        tmp_q_table = np.copy(q_table)
        tmp_q_table[current_state, current_state] = 0

        # if there are no exiting edges, path does not exist
        if max(tmp_q_table[current_state,]) == 0:
            print(f'Path does not exit from {start_location} to {end_location}')
            return 0
        next_state = np.argmax(tmp_q_table[current_state,])
        next_location = state_to_location[next_state]
        route.append(next_location)
        current_location = next_location

    print(route)

In [37]:
# Define the states
location_to_state = {
    'L1': 0,
    'L2': 1,
    'L3': 2,
    'L4': 3,
    'L5': 4,
    'L6': 5,
    'L7': 6,
    'L8': 7,
    'L9': 8
}

# Define the environment
env = np.array([[0, 1, 1, 0, 0, 0, 0, 0, 0],
                [1, 0, 0, 1, 0, 0, 0, 0, 0],
                [1, 0, 0, 0, 0, 0, 1, 0, 0],
                [0, 0, 0, 0, 1, 1, 0, 0, 0],
                [0, 1, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 1, 0, 0, 0, 0, 1],
                [0, 0, 1, 0, 0, 0, 0, 0, 1],
                [0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 1, 1, 1, 0]])

# Define the actions
actions = [i for i in range(len(env))]

# Create a dictionary mapping from state to location
state_to_location = dict((state, location) for location, state in location_to_state.items())

In [49]:
# set hyperparameters

alpha = 0.9  # learning rate
gamma = 0.75  # Discount factor
iterations = 1000

# define reward
end_location = 'L9'
end_reward = 1000

reward_structure = np.zeros_like(env)
end_state = location_to_state[end_location]
reward_structure[end_state, end_state] = end_reward

# additional reward
'''
prefered_location = 'L4'
prefered_location_reward = 800
prefered_state = location_to_state[prefered_location]
reward_structure[prefered_state, prefered_state] = prefered_location_reward
'''

# combine the reward and env map
reward_env = np.add(env, reward_structure)
print(f'Environment with reward structure:\n {reward_env}')

Environment with reward structure:
 [[   0    1    1    0    0    0    0    0    0]
 [   1    0    0    1    0    0    0    0    0]
 [   1    0    0    0    0    0    1    0    0]
 [   0    0    0    0    1    1    0    0    0]
 [   0    1    0    0    0    0    0    0    0]
 [   0    0    0    1    0    0    0    0    1]
 [   0    0    1    0    0    0    0    0    1]
 [   0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    1    1    1 1000]]


In [50]:
# Train

# Q table will be printed when training iteration = check_point
check_point = 1000

q_table = train(alpha, gamma, iterations, location_to_state, state_to_location, actions, reward_env, check_point)

Q-table at iteration 1000:
 [[   0 1264 1686    0    0    0    0    0    0]
 [1265    0    0 1685    0    0    0    0    0]
 [1265    0    0    0    0    0 2247    0    0]
 [   0    0    0    0  948 2247    0    0    0]
 [   0 1264    0    0    0    0    0    0    0]
 [   0    0    0 1685    0    0    0    0 2996]
 [   0    0 1686    0    0    0    0    0 2996]
 [   0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0 2247 2244    0 3994]]



In [51]:
# Print shortest path between any two locations
start_location = 'L1'

get_optimal_route(start_location, end_location, q_table, location_to_state, state_to_location)

['L1', 'L3', 'L7', 'L9']
